In [ ]:
import ipyparallel
from tempfile import TemporaryFile
data = TemporaryFile()    

In [ ]:
clients = ipyparallel.Client('/home/snara025/.ipython/profile_myprofile/security/ipcontroller-client.json')#, url_file='/home/snara025/.ipython/profile_myprofile/security/ipcontroller-client.json', sshserver='192.168.233.126',
clients.ids

lview = clients.load_balanced_view() 
dview = clients[:]

In [ ]:
%%px --local

from __future__ import print_function
from __future__ import division
from builtins import str
from builtins import range
from os.path import join as opj

from nipype.interfaces.dcm2nii import Dcm2niix # to convert Dicom to Niftii


import os                                    # system functions
import sys
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.fsl as fsl          # fsl
import nipype.interfaces.afni as afni
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model generation
import nipype.algorithms.rapidart as ra      # artifact detection

In [ ]:
%%px --local
# define important package paths here
os.system('module load fsl/5.0.10')
os.system('module load afni')
sys.path.append("/home/applications/dcm2niix/build/bin")
sys.path.append("/home/applications/fsl/5.0.10/bin")
sys.path.append("/home/applications/fsl/5.0.10/etc")
sys.path.append('/home/applications/afni/abin')

In [ ]:
os.environ.keys()


## Make directories
* use the following code to create directories based on whatever format you want (e.g. BIDS)


subject_list = ['sub-01','sub-02', 'sub-03', 'sub-04']

for subs in subject_list:
    os.mkdir('/scratch/sanjay/GRT/fmri_data/nifti')
    os.mkdir('/scratch/sanjay/GRT/fmri_data/nifti/{}'.format(subs))
    os.mkdir('/scratch/sanjay/GRT/fmri_data/nifti/{}/anat'.format(subs))
    os.mkdir('/scratch/sanjay/GRT/fmri_data/nifti/{}/func'.format(subs))




# Dicom Convert

* Needs to be done only once

sys.path.append("/home/applications/dcm2niix/build/bin")



for a in range(1,5,1): # in reality this should be from 1 to 4 but I'm skipping 1 cuz it's already done
    convertnode = pe.Node(Dcm2niix(), name ='convert_dicoms', iter_field=['source_dir'])
    convertnode.inputs.source_dir = '/scratch/sanjay/GRT/fmri_data/sessions/session-0{}/' .format(a)

    convertnode.inputs.output_dir = '/scratch/sanjay/GRT/fmri_data/nifti/session-0{}/' .format(a)
    #convertnode.inputs.merge_images = True

    convertnode.run()

# Data Inputs

### This is a super important section. It is where we specify the input structure for nipype. 

* going off http://nipype.readthedocs.io/en/latest/users/grabbing_and_sinking.html http://nipype.readthedocs.io/en/latest/users/examples/fmri_fsl.html

* not sure which T1 to use, we have like 2 for each session
* The purpose of MapNode is to map the node when the node takes only one input. Some regular nodes take more than 1 input. 

In [ ]:
data_dir = '/scratch/sanjay/GRT/fmri_data/nifti/' #  ' '/Users/Narasiwodeyar'
output_dir = '/scratch/sanjay/GRT/fmri_data/testing/output/'
working_dir ='/scratch/sanjay/GRT/fmri_data/testing/output/working' #   '/Users/Narasiwodeyar/Google Drive/Experiments/GRT-fMRI/scripts/'

# list of subject identifiers
subject_list = ['sub-02', 'sub-01']

task_list = ['train_', 'test_']
run_list = ['1','2','3','4','5','6','7','8','9','10','11','12']

In [ ]:
#info = dict(func=[['subject_id', ['run01' ]]], struct=[['subject_id', 't1']])

infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id','task_id','run_id']),name="infosource")
infosource.iterables = [('subject_id', subject_list),('task_id',task_list ),('run_id',run_list)]

## Rename Files

* This works perfectly. What we need is to merge the two sessions from each subject into 1, and reorganize into bids format.

* Note that the following code needs to be changed based on how you want your data. 
* I am gonna call the runs in second session as test_7,train_7, test_8, train_8 etc.

for i in range(1,7,1):

    os.system('mv /scratch/sanjay/GRT/fmri_data/nifti/session-03/*VISUAL_Train_{}.nii  /scratch/sanjay/GRT/fmri_data/nifti/sub-01/func/sub-01_train_{}.nii' .format(i,i)) 
    os.system('mv /scratch/sanjay/GRT/fmri_data/nifti/session-03/*VISUAL_{}.nii /scratch/sanjay/GRT/fmri_data/nifti/sub-01/func/sub-01_test_{}.nii' .format(i,i) )
    
    os.system('mv /scratch/sanjay/GRT/fmri_data/nifti/session-02/*VISUAL_Train_{}.nii  /scratch/sanjay/GRT/fmri_data/nifti/sub-02/func/sub-02_train_{}.nii'.format(i,(i+6))) 
    os.system('mv /scratch/sanjay/GRT/fmri_data/nifti/session-02/*VISUAL_{}.nii /scratch/sanjay/GRT/fmri_data/nifti/sub-02/func/sub-02_test_{}.nii' .format(i,(i+6)))
    
    os.system('mv /scratch/sanjay/GRT/fmri_data/nifti/session-04/*VISUAL_Train_{}.nii  /scratch/sanjay/GRT/fmri_data/nifti/sub-01/func/sub-01_train_{}.nii' .format(i,(i+6))) 
    os.system('mv /scratch/sanjay/GRT/fmri_data/nifti/session-04/*VISUAL_{}.nii  /scratch/sanjay/GRT/fmri_data/nifti/sub-01/func/sub-01_test_{}.nii' .format(i,(i+6)) )
    

### Getting Data
* read this about Data grabbing http://nipype.readthedocs.io/en/latest/users/grabbing_and_sinking.html
* and this http://nipype.readthedocs.io/en/latest/interfaces/generated/nipype.interfaces.io.html

In [ ]:
# Something super important is that the nifti files MUST be uncompressed for this step: nii NOT nii.gz

func_file = opj('{subject_id}', 'func',
                '{subject_id}_{task_id}{run_id}.nii')

anat_file = opj('{subject_id}','anat', '{subject_id}_T1.nii')# we might have to make 3rd iterable for the runs
#resampled_t1 = opj('{subject_id}','anat', '{subject_id}_T1_resampled.nii')
brain = opj('{subject_id}','anat', '{subject_id}_brain.nii')


templates = {'func': func_file, 'anat': anat_file,  't1_brain':brain }
datasource = pe.Node(nio.SelectFiles(templates),name='Get_files')
datasource.inputs.base_directory = data_dir




In [ ]:
fsl.FSLCommand.set_default_output_type('NIFTI')

## Create a workflow

In [ ]:
preproc = pe.Workflow(name='preproc', base_dir=working_dir) # this creates a pre-processing workflow of sorts. pe is pipleline engine

## do slicetiming 

In [ ]:
slice_timer = pe.MapNode(interface=fsl.SliceTimer(interleaved = True), name= 'Slice_time_correction', iterfield=['in_file'])

## Use FSL McFLIRT to do motion correction

* see http://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.fsl/preprocess.html#mcflirt

In [ ]:
motion_correct = pe.MapNode(interface=fsl.MCFLIRT(save_mats=True,
                                                  save_plots=True, interpolation = 'sinc', output_type='NIFTI', dof =6),
                            name='Realign_with_MCFLIRT',
                            iterfield=['in_file'])



In [ ]:
# plot and save motion corrected plots

plot_motion = pe.MapNode(interface=fsl.PlotMotionParams(in_source='fsl'),
                         name='Check_motion_plots',
                         iterfield=['in_file'])
plot_motion.iterables = ('plot_type', ['rotations', 'translations'])


### Strip skull

* You need a skull stripped T1 to do registration.
* I used the skull stripped T1 from recon-all ouput

In [ ]:
#strip = pe.MapNode(interface = fsl.BET(output_type='NIFTI', functional=False), name='Strip_skull_T1', iterfield=['in_file'])

### co-registration to structural needs T1 to be in same space

* This has been achieved using AFNI's 3d-resample. 
* I did this in the command line because I had trouble getting afni to work properly in the parallel engines
* Besides this only needs to be done once

In [ ]:


#resample = pe.Node(interface = afni.Resample(outputtype= 'NIFTI', voxel_size=(2.4,2.4,2.4)), name= 'Resample_T1')

## Select the average volume as input for FLIRT

In [ ]:
avg_vol = pe.MapNode(interface=fsl.MeanImage(output_type='NIFTI',dimension= 'T'),name='Select_avg_vol' ,iterfield = ['in_file'])

### register 4d functional run using epi_reg 

In [ ]:
#reg = pe.MapNode(interface =fsl.EpiReg(output_type = 'NIFTI'), name= 'Register_4d_funcs', iterfield = ['epi', 't1_brain', 't1_head'])

In [ ]:
reg = pe.MapNode(interface =fsl.FLIRT(output_type = 'NIFTI', cost_func='bbr', dof = 6), name= 'FLIRT_reg', iterfield = ['in_file', 'reference'])

### Get the inverse transform matrix

In [ ]:
inverse = pe.MapNode(interface = fsl.ConvertXFM(invert_xfm=True),name= 'Get_inverse_transform', iterfield=['in_file'])

## Apply transform to functionals

In [ ]:
transform = pe.MapNode(interface = fsl.ApplyXFM(interp='trilinear', output_type = 'NIFTI'), name = 'Transform_func_runs', iterfield = ['in_file'])

## Merge all runs from one subject into 1 file

In [ ]:
#merge = pe.Node(interface = fsl.Merge(dimension = 't'), name= 'Merge_all_runs_per_subject')

## Data storing node

In [ ]:
datasink = pe.Node(nio.DataSink(), name='store_outputs')
datasink.inputs.base_directory = output_dir


## workflow connections 

* Define all workflow connections here

In [ ]:

preproc.connect(datasource, 'func', slice_timer, 'in_file')
preproc.connect(slice_timer, 'slice_time_corrected_file', motion_correct, 'in_file')

preproc.connect(infosource,'subject_id',datasource, 'subject_id')
preproc.connect(infosource,'task_id',datasource, 'task_id')
preproc.connect(infosource,'run_id',datasource, 'run_id')
preproc.connect(motion_correct, 'out_file', datasink, 'motion.corrected_files')
preproc.connect(motion_correct, 'par_file', plot_motion, 'in_file')
preproc.connect(motion_correct, 'par_file', datasink,'motion.par')
preproc.connect(plot_motion, 'out_file', datasink, 'plots')

preproc.connect(motion_correct, 'out_file', avg_vol, 'in_file')
preproc.connect(avg_vol, 'out_file', reg, 'in_file') # input functional files to registrations step
preproc.connect(datasource, 't1_brain', reg, 'reference') # store registered files

preproc.connect(reg, 'out_matrix_file', inverse, 'in_file')
preproc.connect(inverse, 'out_file', transform, 'in_matrix_file' )
preproc.connect(motion_correct, 'out_file', transform, 'in_file')
preproc.connect(datasource, 'anat', transform, 'reference')
preproc.connect(transform, 'out_file', datasink, 'final')




### Visualing workflow
* Use the following cell to generate a visual workflow

In [ ]:

#preproc.write_graph("workflow_graph_mcf.dot", graph2use = 'colored')
#from IPython.display import Image
#Image(filename="workflow_graph_mcf.dot.png")

In [ ]:
preproc.run(plugin='IPython', plugin_args={ 'url_file': '/home/snara025/.ipython/profile_myprofile/security/ipcontroller-client.json'})

par = np.loadtxt('/scratch/sanjay/GRT/fmri_data/testing/output/datasink/motion/par/sub-02/sub-02_train_1_mcf.nii.gz.par1_bold_mcf.par')
fig, axes = plt.subplots(2, 1, figsize=(15, 5))
axes[0].set_ylabel('rotation (radians)')
axes[0].plot(par[0:, :3])
axes[1].plot(par[0:, 3:])
axes[1].set_xlabel('time (TR)')
axes[1].set_ylabel('translation (mm)')